In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from preprocess import preprocess
from sales_processing import fill_in_dates, add_date_features

In [2]:
def wmape(y_true, y_pred):
    return np.abs(y_true - y_pred).sum() / np.abs(y_true).sum()


In [3]:
sales = pd.read_csv('data/caspecoTrainingData.csv', parse_dates=True) 
sales = preprocess(sales, fill_dates=True, drop_cols=['Year'], categorical_features=['Month','Day','Weekday','Week'])

In [4]:
sales = sales.rename(columns={'Sales':'y', 'Date':'ds', 'Company':'unique_id'})


In [5]:
train = sales[sales['ds'] < '2022-05-01']
valid = sales[sales['ds'] >= '2022-05-01']
h = valid['ds'].nunique()

In [6]:
import torch
from typing import Union


class WMAPE(torch.nn.Module):

    def __init__(self):
        super(WMAPE, self).__init__()
        self.outputsize_multiplier = 1
        self.output_names = [""]
        self.is_distribution_output = False

    def domain_map(self, y_hat: torch.Tensor):
        return y_hat.squeeze(-1)

    def __call__(
        self,
        y: torch.Tensor,
        y_hat: torch.Tensor,
        mask: Union[torch.Tensor, None] = None,
    ):
        if mask is None:
            mask = torch.ones_like(y_hat)

        num = mask * (y - y_hat).abs()
        den = mask * y.abs()
        return num.sum() / den.sum()


/home/fknorring/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from neuralforecast import NeuralForecast
from neuralforecast.auto import AutoLSTM
from neuralforecast.losses.pytorch import MAPE

models = [AutoLSTM(h=h,
                   num_samples=30,
                   loss=WMAPE())]

model = NeuralForecast(models=models, freq='D')


In [8]:
model.fit(train)

p = model.predict().reset_index()
p = p.merge(valid[['ds', 'unique_id', 'y']],
            on=['ds', 'unique_id'], how='left')


(train_tune pid=4791) 2023-02-22 11:40:05.004954: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
(train_tune pid=4791) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(train_tune pid=4791) 2023-02-22 11:40:07.091690: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
(train_tune pid=4791) 2023-02-22 11:40:07.092055: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
(train_tune pid=4791) 2023-02-22 11:40:07.092086: W tensorflow/compiler/tf2ten

Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]
Training: 0it [00:00, ?it/s]                                               
Epoch 12:   0%|          | 0/1 [00:00<?, ?it/s, loss=1, v_num=0, train_loss_step=0.999, train_loss_epoch=0.999]        


2023-02-22 11:41:25,520	WARNING tune.py:686 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 


Epoch 14:   0%|          | 0/1 [00:00<?, ?it/s, loss=1, v_num=0, train_loss_step=0.999, train_loss_epoch=0.999]        
